### 0. Import libraries

In [1]:
import torch
from transformers import AutoModel, AutoTokenizer


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Current device: {torch.cuda.current_device()} ({device}) -> {torch.cuda.get_device_name(0)}")

### 1. Load the model and the tokenizer

In [3]:
model_path = 'Alibaba-NLP/gte-base-en-v1.5'
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModel.from_pretrained(model_path, trust_remote_code=True).to(device)

### 2. Embed each input text using a batch of 1

In [4]:
input_texts = [
    "what is the capital of China?",
    "how to implement quick sort in python?",
    "Beijing",
    "sorting algorithms"
] * 100


In [ ]:
for text in input_texts:
    # tokenize the input texts
    batch_dict = tokenizer([text], max_length=8192, padding=True, truncation=True, return_tensors='pt').to(device)

    # get embeddings
    outputs = model(**batch_dict)
    embeddings = outputs.last_hidden_state[:, 0]

    del batch_dict,outputs, embeddings